# laser_visualisation

Visualisation of the depth camera as a laser sensor in a 2D plot.

In [ ]:
# Insérez ici le numéro de votre équipe
# (en) Insert here your team number
VOTRE_NUMERO_EQUIPE = 0

## Import des paquets Python

In [ ]:
# Import des paquets requis et configuration du ROS Master
# (en) Import of required packages and configuration of ROS Master
import sys
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/'); sys.path.append('/usr/lib/python3/dist-packages/')

In [ ]:
from os import environ
import rospy
from nav_msgs.msg import Odometry
from jackal_msgs.msg import Drive
from nav_msgs.msg import OccupancyGrid
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Pose
from scipy.spatial.transform import Rotation as R
import numpy as np
import time

In [ ]:
#environ['ROS_MASTER_URI'] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)
environ['ROS_MASTER_URI'] = "http://localhost:11311/"

In [ ]:
# Créer et démarrer un nouveau noeud
# (en) Create and start a new node
rospy.init_node('laser_visualization', anonymous=True)

## Création des 'callbacks' (Create callbacks)

Chaque fonction est associée à un sujet ROS et enregistre en continue sont contenu dans une variable globale.

(en) Each function is associated with a ROS topic and continuously records its content in a global variable.

In [ ]:
# Laser scan subscriber callback
laser_msg = LaserScan()
def laser_scan_callback(msg):
    global laser_msg
    laser_msg = msg

In [ ]:
# Realsense Pose subscriber callback
pose_msg = Pose()
def pose_callback(msg):
    global pose_msg
    pose_msg  = msg.pose.pose
# Fonction pour le calcul de l'orientation à partir d'un quaternion
# (en) Function for calculating the orientation from a quaternion
def get_heading_from_quaternion(q):
    r = R.from_quat([q.x, q.y, q.z, q.w])
    angles = r.as_euler('xyz', degrees=False)
    return angles[2]
def wraptopi(angle):
    xwrap=np.remainder(angle, 2*np.pi)
    if np.abs(xwrap)>np.pi:
        xwrap -= 2*np.pi * np.sign(xwrap)
    return xwrap

In [ ]:
# ROS subscribers et publishers
laser_scan_sub = rospy.Subscriber('/scan', LaserScan, laser_scan_callback)
pose_sub = rospy.Subscriber('/mobile_manip/t265/odom/sample', Odometry, pose_callback)
cmd_drive_pub = rospy.Publisher('/mobile_manip/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)

## Contrôleur du mouvement (Movement controller)

In [ ]:
interwheel_distance = 0.3765
left_wheel_radius = 0.098
right_wheel_radius = 0.098
def move_robot(linear, angular):
    vel_left  = (linear - angular * interwheel_distance / 2.0) / left_wheel_radius
    vel_right = (linear + angular * interwheel_distance / 2.0) / right_wheel_radius

    # Envoi des commandes au roues par topic ROS
    # (en) Sending commands to the wheels via ROS topic
    cmd_drive_msg = Drive()
    cmd_drive_msg.drivers[0] = vel_left
    cmd_drive_msg.drivers[1] = vel_right
    cmd_drive_pub.publish(cmd_drive_msg)

## Interface manuelle (Manual interface)

Permet le contrôle du robot avec des flèches et un retour visuel des obstacles vu par la D435.

(en) Allows the robot to be controlled with arrows and a visual feedback of the obstacles seen by the D435.

In [ ]:
%matplotlib ipympl
import ipywidgets as widgets
import threading
import math
import matplotlib.pyplot as plt

btn_up = widgets.Button(icon='arrow-up')
btn_left = widgets.Button(icon='arrow-left')
btn_down = widgets.Button(icon='arrow-down')
btn_right = widgets.Button(icon='arrow-right')
btn_stop = widgets.Button(description='Arret')

fig = plt.figure(figsize=(6,3))

def update_plot():
    global fig
    while True:
        cap = wraptopi(get_heading_from_quaternion(pose_msg.orientation))
        x_pos = pose_msg.position.x
        y_pos = pose_msg.position.y
        obstacles_xy = []

        for i, r in enumerate(laser_msg.ranges):
            angle = cap - laser_msg.angle_min + i * laser_msg.angle_increment
            obstacles_xy.append([x_pos + r * math.cos(angle), y_pos + r * math.sin(angle)])
            # Arrête si un obstacle directement à l'avant à moins de 1m.
            # (en) Stop if an obstacle directly in front within 1m.
            if i == 0 and r < 1.0 and r > 0.01:
                move_robot(0,0)

        plt.clf()
        plt.scatter(np.asarray(obstacles_xy)[:,0],np.asarray(obstacles_xy)[:,1], color='r')
        plt.scatter(x_pos,y_pos, color='g')
        plt.axis([-10, 10, -10, 10])
        plt.grid(True)
        fig.canvas.draw()
        plt.pause(0.1)

def on_btn_up_clicked(b):
    move_robot(1,0)

def on_btn_left_clicked(b):
    move_robot(0,1)

def on_btn_down_clicked(b):
    move_robot(-1,0)

def on_btn_right_clicked(b):
    move_robot(0,-1)

def on_btn_stop_clicked(b):
    move_robot(0,0)

btn_up.on_click(on_btn_up_clicked)
btn_left.on_click(on_btn_left_clicked)
btn_down.on_click(on_btn_down_clicked)
btn_right.on_click(on_btn_right_clicked)
btn_stop.on_click(on_btn_stop_clicked)


grid = widgets.GridspecLayout(4, 3,height='500px')

grid[1,1] = btn_up
grid[2,0] = btn_left
grid[2,1] = btn_stop
grid[2,2] = btn_right
grid[3,1] = btn_down
thread = threading.Thread(target=update_plot)
thread.start()
grid